# Tarea 12 - Análisis de sentimientos

# MINERÍA DE TEXTOS

## Alumna: Jessica Sarahi Méndez Rincón

1- Descarga de corpus Movie reviews: http://ai.stanford.edu/~amaas/data/sentiment/

2 - Usar el promedio de palabras en cada review para representar el documento.

3 - Usar doc2vec para obtener el vector del documento.

4 - Elegir uno de los otros métodos vistos para generar el vector de documento.

5 - Verificar que método obtiene mejor exactitud (accuracy) para clasificar los reviews en las etiquetas de sentimiento correspondiente.

### Carga del corpus

In [3]:
from os import listdir
 
# leer el archivo dentro de la memoria
def load_doc(filename):
    # abrir el archivo y leerlo únicamente
    file = open(filename, 'r')
    #leer todo el texto
    text = file.read()
    # cerrar el archivo
    file.close()
    return text 

In [8]:
#Leer todos los documentos en el directorio
def process_docs(directory):
    # recorrer todos los archivos de la carpeta
    for filename in listdir(directory):
        # omitir archivos que no tienen la extensión correcta
        if not filename.endswith(".txt" ):
            continue
        # crear la ruta completa del archivo para abrir
        path = directory + '/' + filename
        # leer el documento
        doc = load_doc(path)
        print('Loaded %s' % filename)
 
#Especificar el directorio a leer
carpeta = 'test/pos'
process_docs(carpeta)

Loaded 0_10.txt
Loaded 10000_7.txt
Loaded 10001_9.txt
Loaded 10002_8.txt
Loaded 10003_8.txt
Loaded 10004_9.txt
Loaded 10005_8.txt
Loaded 10006_7.txt
Loaded 10007_10.txt
Loaded 10008_8.txt
Loaded 10009_10.txt
Loaded 1000_9.txt
Loaded 10010_9.txt
Loaded 10011_9.txt
Loaded 10012_9.txt
Loaded 10013_9.txt
Loaded 10014_7.txt
Loaded 10015_8.txt
Loaded 10016_8.txt
Loaded 10017_8.txt
Loaded 10018_8.txt
Loaded 10019_8.txt
Loaded 1001_10.txt
Loaded 10020_8.txt
Loaded 10021_9.txt
Loaded 10022_10.txt
Loaded 10023_8.txt
Loaded 10024_9.txt
Loaded 10025_8.txt
Loaded 10026_10.txt
Loaded 10027_8.txt
Loaded 10028_10.txt
Loaded 10029_10.txt
Loaded 1002_9.txt
Loaded 10030_7.txt
Loaded 10031_8.txt
Loaded 10032_8.txt
Loaded 10033_8.txt
Loaded 10034_9.txt
Loaded 10035_10.txt
Loaded 10036_10.txt
Loaded 10037_7.txt
Loaded 10038_7.txt
Loaded 10039_8.txt
Loaded 1003_8.txt
Loaded 10040_10.txt
Loaded 10041_8.txt
Loaded 10042_8.txt
Loaded 10043_10.txt
Loaded 10044_8.txt
Loaded 10045_7.txt
Loaded 10046_10.txt
Loaded 

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 1899: character maps to <undefined>

In [9]:
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load the document
filename = 'test/pos/10069_10.txt'
text = load_doc(filename)
# split into tokens by white space
tokens = text.split()
print(tokens)

['I', 'love', 'this', 'movie!!', 'Sure', 'I', 'love', 'it', 'because', 'of', 'Madonna', 'but', 'who', 'cares', '-', "it's", 'damn', 'funny!!!', '*ALANiS', 'Rocks*.', 'When', 'I', 'first', 'saw', 'this', 'film', 'in', 'the', 'theatres', 'back', 'in', '1987,', 'I', 'thought', 'it', 'was', 'all', 'out', 'hilarious!', 'Madonna', 'is', 'so', 'funny', 'and', 'I', 'love', 'her', 'dubbed', 'accent', 'and', 'wacky/funky', 'look.', 'The', 'all-time', 'funniest', 'part', 'is', 'when', 'Madonna(Nikki)', 'screams', 'at', 'a', 'man', 'who', 'is', 'about', 'to', 'get', 'into', 'a', 'taxi.', 'And', 'also', 'when', 'Griffin', 'Dunne(Louden)trips', 'and', 'falls', 'at', 'the', 'apartment', 'interview', 'scene.', '**ALANiS', 'Rocks**.', "Madonna's", 'character', 'Nikki', 'steals/shop', 'lifts', 'and', 'fools', 'people', 'throughout', 'the', 'whole', 'movie', '-', 'her', 'hilarious', 'antics', 'are', 'enough', 'to', 'keep', 'you', 'on', 'the', 'floor', 'the', 'whole', 'time.', '"Didn\'t', 'rob', "nothin',

### Vocabulario

In [18]:

from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r', encoding="utf8")
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens
 
# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
	# load doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# update counts
	vocab.update(tokens)
 
# leer todos los directorios en el documento
def process_docs(directory, vocab):
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip files that do not have the right extension
		if not filename.endswith(".txt"):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# add doc to vocab
		add_doc_to_vocab(path, vocab)
 
# save list to file
def save_list(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w', encoding="utf8")
	file.write(data)
	file.close()
 
# define vocab
vocab = Counter()
# add all docs to vocab
process_docs('train/neg', vocab)
process_docs('train/pos', vocab)
# print the size of the vocab
print(len(vocab))
# print the top words in the vocab
print(vocab.most_common(50))
# keep tokens with > 5 occurrence
min_occurane = 5
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))
# save tokens to a vocabulary file
save_list(tokens, 'vocab.txt')

138430
[('br', 57141), ('The', 44351), ('movie', 41198), ('film', 37009), ('one', 23188), ('like', 18876), ('This', 14725), ('good', 13927), ('It', 12074), ('would', 12010), ('time', 11504), ('really', 11276), ('story', 11058), ('even', 10882), ('see', 10854), ('much', 9289), ('get', 8996), ('people', 8474), ('bad', 8456), ('great', 8260), ('made', 7887), ('first', 7881), ('well', 7850), ('also', 7722), ('make', 7599), ('films', 7597), ('movies', 7587), ('could', 7567), ('way', 7467), ('dont', 7333), ('But', 7216), ('characters', 7183), ('think', 7078), ('Its', 6738), ('And', 6715), ('seen', 6486), ('character', 6455), ('watch', 6269), ('many', 6233), ('two', 6171), ('plot', 6137), ('acting', 6110), ('never', 6101), ('little', 5981), ('know', 5954), ('In', 5880), ('best', 5717), ('love', 5701), ('show', 5683), ('life', 5651)]
35629


### Extracción del corpus desde el sitio
#### gensim doc2vec & IMDB sentiment dataset

In [47]:
import os.path
assert os.path.isfile("alldata-id-10.txt"), "alldata-id.txt unavailable"

In [48]:
import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple

SentimentDocument = namedtuple('SentimentDocument', 'words tags split sentiment')

alldocs = []  # will hold all docs in original order
with open('alldata-id-10.txt', encoding='utf-8') as alldata:
    for line_no, line in enumerate(alldata):
        tokens = gensim.utils.to_unicode(line).split()
        words = tokens[1:]
        tags = [line_no] # `tags = [tokens[0]]` would also work at extra memory cost
        split = ['train','test','extra','extra'][line_no//25000]  # 25k train, 25k test, 25k extra
        sentiment = [1.0, 0.0, 1.0, 0.0, None, None, None, None][line_no//12500] # [12.5K pos, 12.5K neg]*2 then unknown
        alldocs.append(SentimentDocument(words, tags, split, sentiment))

train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']
doc_list = alldocs[:]  # for reshuffling per pass

print('%d docs: %d train-sentiment, %d test-sentiment' % (len(doc_list), len(train_docs), len(test_docs)))

10 docs: 10 train-sentiment, 0 test-sentiment


### Configurar modelos de evaluación y capacitación de Doc2Vec 

Las opciones de parámetros siguientes varían:

Vectores de 100 dimensiones,  el submuestreo frecuente de palabras parece disminuir la precisión de la predicción de sentimientos, por lo que se omite
cbow=0 significa salto-gramo que es equivalente al modo 'PV-DBOW' del papel, emparejado en gensim con dm=0
A ese modelo DBOW se le agregan dos modelos DM, uno que promedia los vectores de contexto ( dm_mean) y otro que los concatena (lo que dm_concatda como resultado un modelo mucho más grande, más lento y más hambriento de datos)
a min_count=2ahorra bastante memoria del modelo, descartando solo las palabras que aparecen en un solo documento (y, por lo tanto, no son más expresivas que los propios vectores de documentos únicos para cada uno)

In [49]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

cores = multiprocessing.cpu_count()
#assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

simple_models = [
    # PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DBOW 
    Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=100, window=10, negative=5, hs=0, min_count=2, workers=cores),
]

# speed setup by sharing results of 1st model's vocabulary scan
simple_models[0].build_vocab(alldocs)  # PV-DM/concat requires one special NULL word so it serves as template
print(simple_models[0])
for model in simple_models[1:]:
    model.reset_from(simple_models[0])
    print(model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t4)
Doc2Vec(dbow,d100,n5,mc2,s0.001,t4)
Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t4)


C:\Users\Jess\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
C:\Users\Jess\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
C:\Users\Jess\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [39]:
!pip install testfixtures

You should consider upgrading via the 'c:\users\jess\anaconda3\python.exe -m pip install --upgrade pip' command.


In [50]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[2]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[0]])

# Métodos de evaluación predictiva 
Métodos auxiliares para evaluar la tasa de error.

In [51]:
import numpy as np
import statsmodels.api as sm
from random import sample

# for timing
from contextlib import contextmanager
from timeit import default_timer
import time 

@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    
def logistic_predictor_from_data(train_targets, train_regressors):
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    #print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set, infer=False, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets, train_regressors = zip(*[(doc.sentiment, test_model.docvecs[doc.tags[0]]) for doc in train_set])
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_data = test_set
    if infer:
        if infer_subsample < 1.0:
            test_data = sample(test_data, int(infer_subsample * len(test_data)))
        test_regressors = [test_model.infer_vector(doc.words, steps=infer_steps, alpha=infer_alpha) for doc in test_data]
    else:
        test_regressors = [test_model.docvecs[doc.tags[0]] for doc in test_docs]
    test_regressors = sm.add_constant(test_regressors)
    
    # predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_data])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

# Entrenamiento 

In [52]:
from collections import defaultdict
best_error = defaultdict(lambda :1.0)  

In [53]:
doc_list

[SentimentDocument(words=['it', 'was', 'a', 'bit', 'bizarre', 'and', 'evil', 'and', 'i', 'enjoyed', 'it', 'a', 'lot', ',', 'the', 'characters', 'in', 'the', 'show', 'were', 'great', 'as', 'well', ',', 'and', 'complimented', 'one', 'another', 'well', '.', 'i', 'was', 'sorry', 'to', 'see', 'it', 'cut', 'off', '.', '.', 'i', 'would', 'have', 'loved', 'to', 'see', 'where', 'it', 'could', 'have', 'went', '.', 'you', 'found', 'yourself', 'leaning', 'toward', 'lucas', 'buck', 'the', 'sheriff', 'who', 'had', 'more', 'secrets', 'than', 'anyone', '.', 'lucas', 'was', 'frightening', 'and', 'alluring', '.', 'and', 'i', 'would', 'have', 'liked', 'to', 'have', 'seen', 'more', 'of', 'him', 'and', 'how', 'his', 'character', 'became', '.', 'i', 'will', 'however', 'buy', 'the', 'show', 'just', 'to', 'enjoy', ',', 'it', 'was', 'great', 'to', 'something', 'different', 'on', 'tv', '.', 'and', 'paige', 'turrco', 'who', 'was', "caleb's", 'cousin', ',', 'she', 'was', 'a', 'big', 'mystery', 'as', 'to', 'where'

In [76]:
from random import shuffle
import datetime

alpha, min_alpha, passes = (0.025, 0.001, 20)
alpha_delta = (alpha - min_alpha) / passes

print("START %s" % datetime.datetime.now())

for epoch in range(passes):
    shuffle(doc_list)  # shuffling gets best results
    
    for name, train_model in models_by_name.items():
        # train
        duration = 'na'
        train_model.alpha, train_model.min_alpha = alpha, alpha
        
        with elapsed_timer() as elapsed:
            train_model.train(doc_list)
            #duration = '%.1f' % elapsed()
            
        # evaluate
        eval_duration = ''
        with elapsed_timer() as eval_elapsed:
            err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs)
        eval_duration = '%.1f' % eval_elapsed()
        best_indicator = ' '
        if err <= best_error[name]:
            best_error[name] = err
            best_indicator = '*' 
        print("%s%f : %i passes : %s %ss %ss" % (best_indicator, err, epoch + 1, name, duration, eval_duration))

        if ((epoch + 1) % 5) == 0 or epoch == 0:
            eval_duration = ''
            with elapsed_timer() as eval_elapsed:
                infer_err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs, infer=True)
            eval_duration = '%.1f' % eval_elapsed()
            best_indicator = ' '
            if infer_err < best_error[name + '_inferred']:
                best_error[name + '_inferred'] = infer_err
                best_indicator = '*'
            print("%s%f : %i passes : %s %ss %ss" % (best_indicator, infer_err, epoch + 1, name + '_inferred', duration, eval_duration))

    print('completed pass %i at alpha %f' % (epoch + 1, alpha))
    alpha -= alpha_delta
    
print("END %s" % str(datetime.datetime.now()))

START 2020-08-25 14:15:05.650435


ValueError: You must specify either total_examples or total_words, for proper job parameters updationand progress calculations. The usual value is total_examples=model.corpus_count.

No se ejecuta por el problema del coteo de palabras para que el job cuente con una ejecución inicial y final.
Ya no lo logré arreglar.

## Intente con otro ejemplo:

In [70]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
data = ["I love hot chocolate.",
        "I hate hot chocolate",
       "I love hot tea.",
       "I love hot cake."]

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
print(tagged_data)
#Train and save
max_epochs = 10
vec_size = 5
alpha = 0.025


model = Doc2Vec(vector_size=vec_size, #it was size earlier
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    if epoch % 10 == 0:
        print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs) #It was model.iter earlier
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

print("Model Ready")

test_sentence="I adore hot chocolate"
test_data = word_tokenize(test_sentence.lower())
v1 = model.infer_vector(test_data)
#print("V1_infer", v1)

# to find most similar doc using tags
sims = model.docvecs.most_similar([v1])
print("\nTest: %s\n" %(test_sentence))
for indx, score in sims:
    print("\t(score: %.4f) %s" %(score, data[int(indx)]))

[TaggedDocument(words=['i', 'love', 'hot', 'chocolate', '.'], tags=['0']), TaggedDocument(words=['i', 'hate', 'hot', 'chocolate'], tags=['1']), TaggedDocument(words=['i', 'love', 'hot', 'tea', '.'], tags=['2']), TaggedDocument(words=['i', 'love', 'hot', 'cake', '.'], tags=['3'])]
iteration 0
Model Ready

Test: I adore hot chocolate

	(score: 0.6380) I hate hot chocolate
	(score: 0.5394) I love hot chocolate.
	(score: 0.2405) I love hot tea.
	(score: 0.1522) I love hot cake.
